# Overwatch Battletag Scraper

### Imports

In [1]:
from bs4 import BeautifulSoup as bs
import urllib2
from pathlib2 import Path
import codecs

### Files

In [2]:
discovery_file_str = "../data/discovery.csv"
discovery_curr_page_file_str = "../data/discovery_curr_page.txt"

### Methods

In [3]:
def update_curr_page_file(file_str):
    """Writes the current page file to the discovery current page file."""
    f = open(file_str, "w")
    f.write(str(curr_page))
    f.close()

In [4]:
import sys
reload(sys)
sys.setdefaultencoding("utf8")

### Scraping

In [4]:
#Discover competitive Overwatch players and scrape their info
#for more detailed scraping at a later date.
tracker_str = "https://overwatchtracker.com/leaderboards/pc/global/CompetitiveRank?page={}&mode=1"

#Get page where parsing left off last
curr_page = -1
try:
    path = Path(discovery_curr_page_file_str).resolve()
except:
    print("Current page file NOT FOUND. Using current page value of 1.")
    curr_page = 1
else:
    discovery_curr_page_file = open(discovery_curr_page_file_str, "r")
    curr_page = int(discovery_curr_page_file.readline())
    print("Current page file FOUND. Resuming scraping from page {}".format(curr_page))
    discovery_curr_page_file.close()

Current page file FOUND. Resuming scraping from page 1647


In [5]:
#Store battle tag tuple info here
battletags = []
#Writing condition
write_every_n_pages = 3
#Stopping condition
no_sr_found = False

#Parse Overwatch Tracker
print("Starting scraping ...")
print("Writing players to file every {} pages ...".format(write_every_n_pages))
while curr_page < 3000:
    #Log
    print("Scraping page {} ...".format(curr_page))
    
    #Below request line assigns a header to avoid getting banned xd
    req = urllib2.Request(tracker_str.format(curr_page), headers={"User-Agent": "Mozilla/5.0"})
    soup = bs(urllib2.urlopen(req).read())
    battletag_htmls = soup.find_all("a", {"data-tooltip": "notooltip"})
    for html in battletag_htmls:
        #End of placed players list (people who've started placement matches but haven't finished)
        if(html.find_next_sibling().getText() == ""):
            no_sr_found = True
            break;
            
        #Add battletag entry (last entry means 0=not scraped, 1= scraped)
        battletags.append((html.parent.previous_sibling.find_previous_sibling().getText(),
                           html.getText(),
                           html.find_next_sibling().getText(),
                           0))       
            
    #Stop scraping condition
    if no_sr_found:
        print("Scraping completed on page {}".format(curr_page))
        #Write last page scraped to file
        update_curr_page_file(discovery_curr_page_file_str)
        break;
    
    #Update current page
    curr_page += 1
    
    #Save data to file every n pages
    if curr_page % write_every_n_pages == 0:
        print("Writing battletags ...")
        #Update discovery file
        with codecs.open(discovery_file_str, "a", encoding="utf8") as f:
            for b in battletags:
                f.write(str(b[0]) + "," + str(b[1]) + "," + str(b[2]) + "," + str(b[3]) + "\n")
        #Clear out discovery
        battletags = []
        print("Battletags written!!")
        
        #Update current page file
        print("Writing current page ...")
        update_curr_page_file(discovery_curr_page_file_str)
        print("Current page written!!")